In [2]:
import os
import csv
from tqdm import tqdm
import tweepy
import pandas as pd
import time
from datetime import datetime
import nltk


CONSUMER_KEY='DfMnPVzPIRS1jyhwEEtIWVNYF'
CONSUMER_SECRET='3YVosIbf8GBGahVCMfjHEVNRBUGb00CI5N4OVkzelwmM5jXVbn'
ACCESS_KEY='1640818273-xsgeYxwxsz5T1FCrE1OhJnR6lvURfai1ija02DW'
ACCESS_SECRET='DNISI77zYk4IN4znu29VcjkPLRKgSWsnXua1i3XjtPlaE'

In [3]:
auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY,ACCESS_SECRET)

In [4]:
# FEATURE DATABASE SCRIPT 

def get_tweets(feature_keywords, num_tweets=100, f_text="extended", retweets="retweeted_status"):

    start_date = "202111010900"  # 03-01-2020 at 9:00 am
    end_date = "202112010900"  # 04-01-2020 at 9:00 am
    api = tweepy.API(auth, wait_on_rate_limit=True)

    # THE PARAMETER LABEL IN SEARCH_FULL_ARCHIVE WILL CHANGE ACCORDING TO YOUR NAMING CONVENTION
    # Create a dev environment in one of your apps: https://developer.twitter.com/en/account/environments
    fetch_feature_tweets = api.search_full_archive(label="darin", query=feature_keywords, fromDate=start_date, toDate=end_date,
                                       maxResults=num_tweets) 

    all_feat_tweets = []
    for ind, tweet in enumerate(tqdm(fetch_feature_tweets)):
        # start_scrape = time.time()
        status = api.get_status(tweet.id, tweet_mode=f_text)
        if hasattr(status, retweets):
            full_text = status.retweeted_status.full_text.encode("utf-8").lower()
            feat_tweets = [tweet.user.screen_name, full_text, tweet.retweeted_status.user.screen_name,
                           tweet.entities['hashtags'], tweet.created_at, tweet.user.followers_count,
                           tweet.user.friends_count, tweet.lang]
            all_feat_tweets.append(feat_tweets)
        else:
            full_text = status.full_text.encode("utf-8").lower()
            feat_tweets = [tweet.user.screen_name, full_text, "no retweet username", tweet.entities['hashtags'],
                           tweet.created_at, tweet.user.followers_count, tweet.user.friends_count, tweet.lang]
            all_feat_tweets.append(feat_tweets)

        print("...{} tweets downloaded so far".format(len(all_feat_tweets)))
        # end_scrape = time.time()
        # total_scrape_time = round((end_scrape - start_scrape) / 60, 2)
        # print('time taken for scrape is {} minutes'.format(total_scrape_time))

    # THIS LINE BELOW COMMENT OUT AFTER YOU'VE USED IT ONCE
    #csv_timestamp = datetime.now().strftime("%m_%d_%Y")
    path = os.getcwd()
    # ALSO COMMENT OUT CSV_TIMESTAMP AFTER YOU'VE RAN THE SCRIPT ONCE
    filename = path + '/' + '2021samplevaccine.csv'

    # FOR YOUR SECOND AND ONGOING TIMES YOU RUN THE SCRIPT, CHANGE W TO A IN THE WITH OPEN LINE
    with open(filename, 'w') as firstcsv:
        writer = csv.writer(firstcsv)
        writer.writerow(["username", "tweet", "username_of_retweeter", "hashtag", "tweet created",
                         "number of followers", "# following"])
        writer.writerows(all_feat_tweets)


if __name__ == "__main__":
    f_keywords = "Pfizer"
    get_tweets(f_keywords)

#features_df = pd.read_csv("bpd_feature_data.csv", header=0)
#features_df.head()
#print(features_df.shape)
#print(features_df.tweet[0])

TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.

In [23]:
# LABEL DATABASE SCRIPT

def get_label_tweets(label_keywords, num_tweets=100, f_text="extended", retweets="retweeted_status"):

    start_date = "202111010900"  # after the first moenth change to 02
    end_date = "202112010900"  # change to 03 going all the way up until 12
    api = tweepy.API(auth, wait_on_rate_limit=True)

    fetch_label_tweets = api.search_full_archive(label="darin", query=label_keywords, fromDate=start_date, toDate=end_date,
                                                 maxResults=num_tweets)

    bigram_phrases = ['Pfizer vaccine', 'Pfizer shot', 'Pfizer jab',
                      'Pfizer’s vaccine', 'Pfizer boost', 'Pfizer jabs', 'Pfizer drug', 'Pfizer trail',
                      '#Pfizer vaccine', 'Pfizer dose', '1st Pfizer', '2nd Pfizer']

    trigram_phrases = ['Pfizer mRNA vaccine', 'Pfizer COVID vaccine', 'Pfizer trail data',
                       'Pfizer Covid-19 vaccines',
                       '1st Pfizer jab', '1st Pfizer shot', '2nd Pfizer jab', '2nd Pfizer shot', '1st Pfizer dose',
                       '2nd Pfizer dose', 'Pfizer COVID jab', 'dose of Pfizer']

    all_label_tweets = []
    for i, tweet in enumerate(tqdm(fetch_label_tweets)):
        # start_scrape = time.time()
        status = api.get_status(tweet.id, tweet_mode=f_text)
        if hasattr(status, retweets):

            full_text = status.retweeted_status.full_text.lower()
            retweet_username = tweet.retweeted_status.user.screen_name

            nltk_tokens = nltk.word_tokenize(full_text)
            bi_grams = nltk.ngrams(nltk_tokens, 2)
            bigram_labels = [bigram for bigram in bi_grams]
            concat_bigrams = [s1 + " " + s2 for s1, s2 in bigram_labels]

            tri_grams = nltk.ngrams(nltk_tokens, 3)
            trigram_labels = [trigram for trigram in tri_grams]
            concat_trigrams = [s1 + " " + s2 + " " + s3 for s1, s2, s3 in trigram_labels]

            label_bigrams = [lbl for lbl in concat_bigrams if lbl in bigram_phrases]
            label_trigrams = [lbl for lbl in concat_trigrams if lbl in trigram_phrases]
            new_labels = label_bigrams + label_trigrams

            label_tweets = [tweet.user.screen_name, full_text, retweet_username, tweet.entities['hashtags'],
                            tweet.created_at, tweet.user.followers_count, tweet.user.friends_count, new_labels]

            all_label_tweets.append(label_tweets)
        else:
            full_text = status.full_text.lower()
            nltk_tokens = nltk.word_tokenize(full_text)
            bi_grams = nltk.ngrams(nltk_tokens, 2)
            bigram_labels = [bigram for bigram in bi_grams]
            concat_bigrams = [s1 + " " + s2 for s1, s2 in bigram_labels]

            tri_grams = nltk.ngrams(nltk_tokens, 3)
            trigram_labels = [trigram for trigram in tri_grams]
            concat_trigrams = [s1 + " " + s2 + " " + s3 for s1, s2, s3 in trigram_labels]

            label_bigrams = [lbl for lbl in concat_bigrams if lbl in bigram_phrases]
            label_trigrams = [lbl for lbl in concat_trigrams if lbl in trigram_phrases]
            new_labels = label_bigrams + label_trigrams

            label_tweets = [tweet.user.screen_name, full_text, "no retweet username", tweet.entities['hashtags'],
                           tweet.created_at, tweet.user.followers_count, tweet.user.friends_count, new_labels]
            all_label_tweets.append(label_tweets)

        print("...{} tweets downloaded so far".format(len(all_label_tweets)))


    #csv_timestamp = datetime.now().strftime("%m_%d_%Y")
    path = os.getcwd()
    fname = path + '/' + '2020vaccinelabel.csv' # % csv_timestamp
    with open(fname, 'a', encoding='utf-8') as secondcsv:
        writer = csv.writer(secondcsv)
        writer.writerow(["username", "tweet", "username_of_retweeter", "hashtag", "tweet created",
                         "number of followers", "# following", "label"])
        writer.writerows(all_label_tweets)


if __name__ == "__main__":
    l_keywords = "PfizerBioNTech OR Pfizer OR Pfizervaccine OR PfizerProud OR PfizerBioN"
    get_label_tweets(l_keywords)

# label_df = pd.read_csv("", header=0)
# print(label_df.label)
# print(label_df.shape)
# print(label_df.tweet[0])

  2%|█▋                                                                                | 2/100 [00:00<00:18,  5.22it/s]

...1 tweets downloaded so far
...2 tweets downloaded so far


  4%|███▎                                                                              | 4/100 [00:00<00:17,  5.43it/s]

...3 tweets downloaded so far
...4 tweets downloaded so far


  6%|████▉                                                                             | 6/100 [00:01<00:16,  5.86it/s]

...

...95 tweets downloaded so far
...96 tweets downloaded so far


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:18<00:00,  5.99it/s]

...97 tweets downloaded so far
...98 tweets downloaded so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.38it/s]

...99 tweets downloaded so far
...100 tweets downloaded so far


In [26]:
label_df = pd.read_csv("2020vaccinelabel.csv", header=0)
print(label_df.label)
print(label_df.shape)
print(label_df.tweet[0])

0       []
1       []
2       []
3       []
4       []
        ..
1105    []
1106    []
1107    []
1108    []
1109    []
Name: label, Length: 1110, dtype: object
(1110, 8)
en israel los expertos locales informan que la vacuna se pfizer tienen mayor mortalidad que el covid. noticia vía le soir aquí en inglés https://t.co/h7zdfzcld5


In [27]:
features_df = pd.read_csv("2020vaccine.csv", header=0)
print(features_df.shape)
print(features_df.tweet[0])

(1009, 7)
b'coronavirus hoy, noticias en directo |  biontech-pfizer prometen hasta 75 millones de vacunas adicionales para la uni\xc3\xb3n europea https://t.co/pkf9ayygdv'


In [28]:
features_df.head()

,username,tweet,username_of_retweeter,hashtag,tweet created,number of followers,# following
0,manuelm70649483,"b'coronavirus hoy, noticias en directo | bion...",no retweet username,[],2021-02-01 08:59:57+00:00,1560,1816
1,iordananutarus1,b'#mironews las eua (emergency use authorizati...,FLMIRONES,"[{'text': 'MIRONEWS', 'indices': [15, 24]}]",2021-02-01 08:59:56+00:00,151,1146
2,stopik,b'izrael zaszczepi\xc5\x82 pierwsz\xc4\x85 daw...,Bob_Gedron,[],2021-02-01 08:59:55+00:00,462,3009
3,PascalRtl,b'coronavirus: pfizer/biontech pr\xc3\xa9voit ...,no retweet username,[],2021-02-01 08:59:53+00:00,10,0
4,luizvagu,"b'#ahora \nsagasti: ""hemos hecho gestiones par...",martinluma,"[{'text': 'AHORA', 'indices': [16, 22]}]",2021-02-01 08:59:52+00:00,98,447


In [29]:
label_df

,username,tweet,username_of_retweeter,hashtag,tweet created,number of followers,# following,label
0,puigdramon,en israel los expertos locales informan que la...,Oespinar1,[],2021-03-01 08:59:59+00:00,290,926,[]
1,YawGyebi,the oxford-astrazeneca vaccine for covid-19 is...,ckgyimah,[],2021-03-01 08:59:55+00:00,1387,1337,[]
2,chescaferrer25,none of us signed up for sinovac. we were prom...,BuffHufflepuff,[],2021-03-01 08:59:49+00:00,575,561,[]
3,Humberto72,#thread sui vaccini #covid mooolto lungo:\r\n\...,Linus2k,"[{'text': 'Thread', 'indices': [13, 20]}, {'te...",2021-03-01 08:59:44+00:00,468,1485,[]
4,minxyminx1952,"had 1st pfizer jab 2 weeks ago. no issues, exc...",no retweet username,[],2021-03-01 08:59:44+00:00,4804,4218,[]
...,...,...,...,...,...,...,...,...
1105,aleveglio,"lui è enrique ruiz, assessore alla sanità dell...",Anna54349379,[],2021-12-01 08:58:30+00:00,31,153,[]
1106,sophora59,@virgilerichard3 @yvespdb heiko von der leyen ...,no retweet username,[],2021-12-01 08:58:29+00:00,25,17,[]
1107,bugbugbugbug11,pfizer blocked marjorie taylor greene on twitt...,JDCocchiarella,[],2021-12-01 08:58:25+00:00,7,145,[]
1108,Air2Aliah,@kingguyo @daniiiiv just know the vaccines are...,no retweet username,[],2021-12-01 08:58:25+00:00,3,36,[]
